Pobierz dane dotyczące scoringu kredytowego

*Pracujesz jako analityk danych w globalnej firmie finansowej. Przez lata firma gromadziła podstawowe dane bankowe i wiele informacji związanych z kredytami. Kierownictwo chce zbudować inteligentny system do segregowania ludzi w przedziałach oceny kredytowej, aby zmniejszyć wysiłek ręczny.*

**Zadanie na dziś: przygotować dane do modelu**

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

import danych

In [2]:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train.csv"

# Load the latest version
df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,
)

print("First 5 records:", df.head())

<ipython-input-2-db52f5663265>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,


100%|██████████| 6.09M/6.09M [00:00<00:00, 147MB/s]

Extracting zip of train.csv...


First 5 records:        ID Customer_ID     Month           Name   Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265  Scientist   

  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0      19114.12            1824.843333                  3  ...           _   
1      19114.12                    NaN                  3  ...        Good   
2      19114.12                    NaN                  3  ...        Good   
3      19114.12                    NaN                  3  ...        Good   
4      19114.12            1824.843333                  3  ...        Good   

   Outstanding_Debt Credit_Utiliz

/usr/local/lib/python3.11/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


**Poznaj dane:**

1.   sprawdź wymiary i typy kolumn
2.   usuń duplikaty
3.   usuń zdbędne kolumny
4.   sprawdz liczbę braków danych w poszczególnych kolumnach






In [6]:
# ad.1
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [7]:
# ad.2
df = df.drop_duplicates()
print("\nLiczba duplikatów po usunięciu:", df.duplicated().sum())


Liczba duplikatów po usunięciu: 0


Usuń kolumny, które nie będą wykorzystywane w analizie ['ID','Customer_ID','SSN','Name']

In [9]:
# ad.3
kolumny_do_usuniecia = ["ID", "Customer_ID", "Name", "SSN"]
df = df.drop(columns=kolumny_do_usuniecia, errors='ignore')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  object 
 1   Age                       100000 non-null  object 
 2   Occupation                100000 non-null  object 
 3   Annual_Income             100000 non-null  object 
 4   Monthly_Inhand_Salary     84998 non-null   float64
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  object 
 9   Type_of_Loan              88592 non-null   object 
 10  Delay_from_due_date       100000 non-null  int64  
 11  Num_of_Delayed_Payment    92998 non-null   object 
 12  Changed_Credit_Limit      100000 non-null  object 
 13  Num_Credit_Inquiries      98035 non-null   fl

In [10]:
# ad. 4
print("\nBraki danych w poszczególnych kolumnach:")
print(df.isnull().sum())


Braki danych w poszczególnych kolumnach:
Month                           0
Age                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64


usuń niektore z zbędnych znakow ('_','-'), zmień puste komorki na nan

In [25]:
df = df.replace({'_': '', '-': ''}, regex=True)
df.head(20)


,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,NaN,23,9,19114.12,1824.843333,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,0,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,3,312.49408867943663,Good
1,NaN,23,9,19114.12,NaN,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,31.944960,NaN,No,49.574949,118.28022162236736,4,284.62916249607184,Good
2,NaN,500,9,19114.12,NaN,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,5,331.2098628537912,Good
3,NaN,23,9,19114.12,NaN,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,6,223.45130972736786,Good
4,NaN,23,9,19114.12,1824.843333,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,2,341.48923103222177,Good
5,NaN,23,9,19114.12,NaN,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,27.262259,22 Years and 6 Months,No,49.574949,62.430172331195294,0,340.4792117872438,Good
6,NaN,23,9,19114.12,1824.843333,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.3440674122349,6,244.5653167062043,Good
7,NaN,23,9,19114.12,1824.843333,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,23.933795,NaN,No,49.574949,24.785216509052056,2,358.12416760938714,Standard
8,NaN,28,0,34847.84,3037.986667,2,4,6,1,CreditBuilder Loan,...,2,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,6,470.69062692529184,Standard
9,NaN,28,10,34847.84,3037.986667,2,4,6,1,CreditBuilder Loan,...,2,605.03,38.550848,26 Years and 8 Months,No,18.816215,40.39123782853101,1,484.5912142650067,Good


Zmień nazwy miesiecy na zmienne liczbowe zgodnie z kolejnością miesięcy

In [19]:
month_mapping = {
    "January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
    "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12
}

if df['Month'].dtype == 'object':
  df['Month'] = df['Month'].map(month_mapping)
  df['MOnth'].fillna(0, inplace = True)
  df['Month'] = df['Month'].astype(int)
else:
  print("The 'Month' column is not of object type. Check the data")

print(df.head())





The 'Month' column is not of object type. Check the data
   Month  Age  Occupation Annual_Income  Monthly_Inhand_Salary  \
0    NaN   23           5      19114.12            1824.843333   
1    NaN   23           5      19114.12                    NaN   
2    NaN  500           5      19114.12                    NaN   
3    NaN   23           5      19114.12                    NaN   
4    NaN   23           5      19114.12            1824.843333   

   Num_Bank_Accounts  Num_Credit_Card  Interest_Rate Num_of_Loan  \
0                  3                4              3           4   
1                  3                4              3           4   
2                  3                4              3           4   
3                  3                4              3           4   
4                  3                4              3           4   

                                        Type_of_Loan  ...  Credit_Mix  \
0  Auto Loan, CreditBuilder Loan, Personal Loan, ...  ...       

Zakoduj zmienne tekstowe: 'Occupation', 'Type_of_Loan', 'Payment_Behaviour', 'Credit_Mix', użyj LabelEncodera

In [27]:
label_enc = LabelEncoder()
for col in ['Occupation', 'Payment_Behaviour', 'Credit_Mix']:
    df[col] = label_enc.fit_transform(df[col].astype(str))
(df[['Occupation', 'Type_of_Loan']].head())


,Occupation,Type_of_Loan
0,7,"Auto Loan, CreditBuilder Loan, Personal Loan, ..."
1,7,"Auto Loan, CreditBuilder Loan, Personal Loan, ..."
2,7,"Auto Loan, CreditBuilder Loan, Personal Loan, ..."
3,7,"Auto Loan, CreditBuilder Loan, Personal Loan, ..."
4,7,"Auto Loan, CreditBuilder Loan, Personal Loan, ..."


Przelicz kolumne Credit_History_Age na liczbę mięsiecy


In [29]:
def convert_credit_history(value):
    if pd.isna(value):
        return np.nan
    parts = value.split()
    years = int(parts[0]) if 'Years' in parts else 0
    # Check if the 'Months' keyword is present and then extract the month value
    months = int(parts[parts.index('Months') - 1]) if 'Months' in parts else 0
    return years * 12 + months

df['Credit_History_Age'] = df['Credit_History_Age'].apply(convert_credit_history)

In [30]:
# Rozdzielanie wartości po przecinku i usuwanie zbędnych spacji
df['Type_of_Loan'] = df['Type_of_Loan'].str.split(', ')

# Użycie MultiLabelBinarizer do One-Hot Encoding
mlb = MultiLabelBinarizer()
encoded_df = pd.DataFrame(mlb.fit_transform(df['Type_of_Loan']), columns=mlb.classes_)

# Łączenie z oryginalnym DataFrame
df = df.join(encoded_df).drop(columns=['Type_of_Loan'])

print(df.head())

TypeError: 'float' object is not iterable

Zmien inne zmienne na numeryczne

Uzupełnij braki w kolumnie Monthly_Inhand_Salary,  Annual_Income, .. średnią

Narysuj rozkłady zmiennych 'Annual_Income', 'Monthly_Balance'

usuń wartości odstające (w przypadku wieku wszystkie ujemne i te powyzej 100 lat)

Zwizualizuj korelacje między zmiennymi